In [ ]:
 import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# DRIVE_PATH = '/content/gdrive/MyDrive/RL-proj'
DRIVE_PATH = '/content/gdrive/MyDrive/cs182/RL-proj'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
# SYM_PATH = '/content/cs182/RL-proj'
# if not os.path.exists(SYM_PATH):
#   !ln -s $DRIVE_PATH $SYM_PATH

In [ ]:
%tensorflow_version 1.x
!pip install procgen -q
!pip install git+https://github.com/openai/baselines #> ~/pip_install_baselines.log
!pip install mpi4py -q

TensorFlow 1.x selected.
  Cloning https://github.com/openai/baselines to /tmp/pip-req-build-_zf_ze5i
  Running command git clone -q https://github.com/openai/baselines /tmp/pip-req-build-_zf_ze5i
  Created wheel for baselines: filename=baselines-0.1.6-cp37-none-any.whl size=220664 sha256=97e5c59b4a35b09fc0d0c72e2720409549fc1f7e702187c9a8c20549658f22b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-irqms37p/wheels/d8/55/79/53870462a2ec11a80a4edf582cd45f173da693fe317190ee29
Successfully built baselines


In [ ]:
import time, os
import tensorflow as tf
from baselines.ppo2 import ppo2
from baselines.common.models import build_impala_cnn
from baselines.common.mpi_util import setup_mpi_gpus
from procgen import ProcgenEnv
from baselines.common.vec_env import (
    VecExtractDictObs,
    VecMonitor,
    VecFrameStack,
    VecNormalize
)
from baselines.bench import Monitor
from baselines import logger
from mpi4py import MPI
import gym

In [ ]:
os.listdir(DRIVE_PATH+'/log') 
# os.listdir('/content/gdrive/MyDrive')

['PPO_CNN_10lvl',
 'PPO_10lvl',
 'PPO_50lvl',
 'ACER_50lvl',
 'baselines',
 'good_PPO_50lvl']

In [ ]:
tf.reset_default_graph()


In [ ]:
log_dir = DRIVE_PATH+'/log/baselines/'
log_dir = os.path.join(log_dir, time.strftime("%d%m%y_%H:%M:%S", time.localtime()))
os.makedirs(log_dir, exist_ok=True)

logger.configure(dir=log_dir)

# tf.reset_default_graph()

num_levels = 50
venv = ProcgenEnv(num_envs=1, env_name="fruitbot", num_levels=num_levels, distribution_mode="easy")
venv = VecExtractDictObs(venv, "rgb")
venv = VecMonitor(
    venv=venv, filename=None, keep_buf=100,
)
venv = VecNormalize(venv=venv, ob=False)


conv_fn = lambda x: build_impala_cnn(x, depths=[16,32,32], emb_size=256)


total_timesteps = int(2e6)

learning_rate = 5e-4
ent_coef = .01
gamma = .999
lam = .95
nsteps = 256
nminibatches = 8
ppo_epochs = 3
clip_range = .2
use_vf_clipping = None
# comm = MPI.COMM_WORLD
# rank = comm.Get_rank()
model = ppo2.learn(
    env=venv,
    network=conv_fn,
    total_timesteps=total_timesteps,
    # save_interval=0,
    nsteps=nsteps,
    nminibatches=nminibatches,
    lam=lam,
    gamma=gamma,
    noptepochs=ppo_epochs,
    log_interval=1,
    save_interval=100,
    ent_coef=ent_coef,
    # mpi_rank_weight=mpi_rank_weight,
    clip_vf=use_vf_clipping,
    # comm=comm,
    lr=learning_rate,
    cliprange=clip_range,
    update_fn=None,
    init_fn=None,
    vf_coef=0.5,
    max_grad_norm=0.5,
)

Streaming output truncated to the last 5000 lines.
| eplenmean               | 80.1     |
| eprewmean               | -0.96    |
| fps                     | 34       |
| loss/approxkl           | 0.0668   |
| loss/clipfrac           | 0.449    |
| loss/policy_entropy     | 2        |
| loss/policy_loss        | -0.0711  |
| loss/value_loss         | 0.0284   |
| misc/explained_variance | -1.68    |
| misc/nupdates           | 666      |
| misc/serial_timesteps   | 1.7e+05  |
| misc/time_elapsed       | 5.17e+03 |
| misc/total_timesteps    | 1.7e+05  |
--------------------------------------
Stepping environment...
Done.
--------------------------------------
| eplenmean               | 79.5     |
| eprewmean               | -0.91    |
| fps                     | 33       |
| loss/approxkl           | 0.0371   |
| loss/clipfrac           | 0.272    |
| loss/policy_entropy     | 2.34     |
| loss/policy_loss        | -0.0448  |
| loss/value_loss         | 0.0868   |
| misc/explained_varia

In [ ]:
def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [ ]:
sess.close() 

In [ ]:
env_name = "fruitbot"
num_envs = 1
distribution_mode = "easy"
num_levels = 10
timesteps_per_proc = int(5e4)
start_level = 0
test_worker_interval = 0
log_dir = SYM_PATH+'/log/baselines/'
log_dir = os.path.join(log_dir, time.strftime("%d%m%y_%H:%M:%S", time.localtime()))


# from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

is_test_worker = False
test_worker_interval = test_worker_interval

if test_worker_interval > 0:
    is_test_worker = rank % test_worker_interval == (test_worker_interval - 1)

learning_rate = 5e-4
ent_coef = .01
gamma = .999
lam = .95
nsteps = 256
nminibatches = 8
ppo_epochs = 3
clip_range = .2
use_vf_clipping = True

mpi_rank_weight = 0 if is_test_worker else 1
num_levels = 0 if is_test_worker else num_levels

if log_dir is not None:
    log_comm = comm.Split(1 if is_test_worker else 0, 0)
    format_strs = ['csv', 'stdout'] if log_comm.Get_rank() == 0 else []
    logger.configure(comm=log_comm, dir=log_dir, format_strs=format_strs)

logger.info("creating environment")
venv = ProcgenEnv(num_envs=num_envs, env_name=env_name, num_levels=num_levels, start_level=start_level, distribution_mode=distribution_mode)
venv = VecExtractDictObs(venv, "rgb")

venv = VecMonitor(
    venv=venv, filename=None, keep_buf=100,
)

venv = VecNormalize(venv=venv, ob=False)

logger.info("creating tf session")
# setup_mpi_gpus()
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True #pylint: disable=E1101
sess = tf.Session()#config=config)
sess.__enter__()

conv_fn = lambda x: build_impala_cnn(x, depths=[16,32,32], emb_size=256)

logger.info("training")
model = ppo2.learn(
    env=venv,
    network=conv_fn,
    total_timesteps=timesteps_per_proc,
    # save_interval=0,
    nsteps=nsteps,
    nminibatches=nminibatches,
    lam=lam,
    gamma=gamma,
    noptepochs=ppo_epochs,
    log_interval=1,
    ent_coef=ent_coef,
    mpi_rank_weight=mpi_rank_weight,
    clip_vf=use_vf_clipping,
    comm=comm,
    lr=learning_rate,
    cliprange=clip_range,
    update_fn=None,
    init_fn=None,
    vf_coef=0.5,
    max_grad_norm=0.5,
)


Logging to /content/cs182/RL-proj/log/baselines/080521_21:26:16
creating environment
creating tf session
training


ValueError: ignored

In [ ]:
env = None



In [ ]:
policy_net = NOne
model = PPO

In [ ]:
model.learn()